## Create RAG ratings for the aggregate data

## Set-up

### Packages and file paths

In [1]:
# Import required packages
from dataclasses import dataclass
import math
import numpy as np
import os
import pandas as pd
#from statsmodels.stats.weightstats import DescrStatsW

# Set display options
pd.set_option('display.max_rows', 100)

In [2]:
# File paths
@dataclass(frozen=True)
class Paths:
    '''Paths to data and files'''
    survey = '../data/survey_data'
    aggregate = 'aggregate_scores.csv'
    rag = 'aggregate_scores_rag.csv'


paths = Paths()

## Import aggregate scores

In [3]:
data = pd.read_csv(os.path.join(paths.survey, paths.aggregate))
data.tail()

,variable,mean,count,school_lab,year_group_lab,gender_lab,fsm_lab,sen_lab
2137,overall_count,NaN,19.0,School G,All,Boy,All,All
2138,overall_count,NaN,57.0,School G,All,All,FSM,All
2139,overall_count,NaN,37.0,School G,All,All,Non-FSM,All
2140,overall_count,NaN,38.0,School G,All,All,All,SEN
2141,overall_count,NaN,51.0,School G,All,All,All,Non-SEN


## Find weighted mean and SD within each group

In [4]:
def descriptives(values, counts):
    '''
    Calculates:
    * Total sample size (across the schools)
    * Total number of groups without NaN (e.g. schools, areas) used in calc
    * Weighted average of the means
    * Standard deviation of the means
    This normalises weights so they sum 1 (and so they can't all be 0).
    It returns the biased variance and is like a weighted version of np.std().
    For small samples, may want to alter to unbiased variance.
    Based on: https://stackoverflow.com/questions/2413522/weighted-standard-deviation-in-numpy
    Inputs:
    - values - series, to calculate mean and std from
    - counts - series, number of students, used to weight calculations
    Output:
    - result - series with each of the calculations (index is name of calc)
    '''
    # Total sample size
    n_pupils = counts.sum(skipna=True)

    # Total number of groups used in other calc - count number of non-NaN rows
    n_groups = counts.count()

    # Weighted mean
    average = np.average(values, weights=counts)

    # Weighted std
    variance = np.average((values-average)**2, weights=counts)
    std = math.sqrt(variance)

    # Combine into a series
    result = pd.Series(
        [n_pupils, n_groups, average, std],
        index=['total_pupils', 'group_n', 'group_wt_mean', 'group_wt_std'])
    return(result)

In [5]:
groups = ['variable', 'year_group_lab', 'gender_lab', 'fsm_lab', 'sen_lab']
wt_mean = (data
           .groupby(groups)
           .apply(lambda x: descriptives(x['mean'], x['count']))
           .reset_index())
wt_mean

,variable,year_group_lab,gender_lab,fsm_lab,sen_lab,total_pupils,group_n,group_wt_mean,group_wt_std
0,accept_score,All,All,All,All,552.0,7.0,10.003623,0.245610
1,accept_score,All,All,All,Non-SEN,286.0,7.0,9.954545,0.383648
2,accept_score,All,All,All,SEN,213.0,6.0,NaN,NaN
3,accept_score,All,All,FSM,All,279.0,7.0,9.906810,0.422856
4,accept_score,All,All,Non-FSM,All,247.0,7.0,10.121457,0.277748
...,...,...,...,...,...,...,...,...,...
301,wellbeing_score,All,All,Non-FSM,All,198.0,7.0,21.090909,0.342797
302,wellbeing_score,All,Boy,All,All,70.0,5.0,NaN,NaN
303,wellbeing_score,All,Girl,All,All,34.0,3.0,NaN,NaN
304,wellbeing_score,Year 10,All,All,All,169.0,6.0,NaN,NaN


In [6]:
rag = pd.merge(data, wt_mean, how='left', on=groups)
rag

,variable,mean,count,school_lab,year_group_lab,gender_lab,fsm_lab,sen_lab,total_pupils,group_n,group_wt_mean,group_wt_std
0,birth_you_age_score,8.105505,109.0,School A,All,All,All,All,667.0,7.0,8.056222,0.481916
1,autonomy_score,17.628571,70.0,School A,All,All,All,All,443.0,7.0,18.033860,0.362478
2,life_satisfaction_score,5.114035,114.0,School A,All,All,All,All,706.0,7.0,5.025496,0.280757
3,optimism_score,12.154762,84.0,School A,All,All,All,All,511.0,7.0,12.037182,0.304259
4,wellbeing_score,21.475410,61.0,School A,All,All,All,All,430.0,7.0,21.160465,0.311695
...,...,...,...,...,...,...,...,...,...,...,...,...
2137,overall_count,NaN,19.0,School G,All,Boy,All,All,128.0,7.0,NaN,NaN
2138,overall_count,NaN,57.0,School G,All,All,FSM,All,361.0,7.0,NaN,NaN
2139,overall_count,NaN,37.0,School G,All,All,Non-FSM,All,323.0,7.0,NaN,NaN
2140,overall_count,NaN,38.0,School G,All,All,All,SEN,272.0,7.0,NaN,NaN


## Create RAG column based on whether 1SD above or below

<mark>remember the reverse scored things</mark>

In [7]:
# Find 1 SD above and below mean
rag['lower'] = rag['group_wt_mean'] - rag['group_wt_std']
rag['upper'] = rag['group_wt_mean'] + rag['group_wt_std']

In [8]:
# Create RAG column
conditions = [(rag['mean'] <= rag['lower']),
              (rag['mean'] > rag['lower']) & (rag['mean'] < rag['upper']),
              (rag['mean'] >= rag['upper'])]
choices = ['below', 'average', 'above']
rag.loc[:, 'rag'] = np.select(conditions, choices, default=np.nan)

Show some examples

In [9]:
test = rag[
    (rag['variable'] == 'autonomy_score') &
    (rag['year_group_lab'] == 'All') &
    (rag['gender_lab'] == 'All') &
    (rag['fsm_lab'] == 'All') &
    (rag['sen_lab'] == 'All')]
test

,variable,mean,count,school_lab,year_group_lab,gender_lab,fsm_lab,sen_lab,total_pupils,group_n,group_wt_mean,group_wt_std,lower,upper,rag
1,autonomy_score,17.628571,70.0,School A,All,All,All,All,443.0,7.0,18.03386,0.362478,17.671382,18.396338,below
298,autonomy_score,18.134146,82.0,School B,All,All,All,All,443.0,7.0,18.03386,0.362478,17.671382,18.396338,average
595,autonomy_score,18.018519,54.0,School C,All,All,All,All,443.0,7.0,18.03386,0.362478,17.671382,18.396338,average
892,autonomy_score,18.114754,61.0,School D,All,All,All,All,443.0,7.0,18.03386,0.362478,17.671382,18.396338,average
1189,autonomy_score,18.148148,54.0,School E,All,All,All,All,443.0,7.0,18.03386,0.362478,17.671382,18.396338,average
1486,autonomy_score,17.473684,57.0,School F,All,All,All,All,443.0,7.0,18.03386,0.362478,17.671382,18.396338,below
1783,autonomy_score,18.676923,65.0,School G,All,All,All,All,443.0,7.0,18.03386,0.362478,17.671382,18.396338,above


In [10]:
test = data[
    (data['variable'] == 'accept_score') &
    (data['year_group_lab'] == 'Year 10') &
    (data['gender_lab'] == 'All') &
    (data['fsm_lab'] == 'All') &
    (data['sen_lab'] == 'All')]
test

,variable,mean,count,school_lab,year_group_lab,gender_lab,fsm_lab,sen_lab
86,accept_score,10.046512,43.0,School A,Year 10,All,All,All
383,accept_score,10.088235,34.0,School B,Year 10,All,All,All
680,accept_score,9.895833,48.0,School C,Year 10,All,All,All
977,accept_score,9.540541,37.0,School D,Year 10,All,All,All
1274,accept_score,10.090909,33.0,School E,Year 10,All,All,All
1571,accept_score,NaN,NaN,School F,Year 10,All,All,All
1868,accept_score,10.540541,37.0,School G,Year 10,All,All,All


In [11]:
test = rag[
    (rag['variable'] == 'accept_score') &
    (rag['year_group_lab'] == 'Year 10') &
    (rag['gender_lab'] == 'All') &
    (rag['fsm_lab'] == 'All') &
    (rag['sen_lab'] == 'All')]
test

,variable,mean,count,school_lab,year_group_lab,gender_lab,fsm_lab,sen_lab,total_pupils,group_n,group_wt_mean,group_wt_std,lower,upper,rag
86,accept_score,10.046512,43.0,School A,Year 10,All,All,All,232.0,6.0,NaN,NaN,NaN,NaN,nan
383,accept_score,10.088235,34.0,School B,Year 10,All,All,All,232.0,6.0,NaN,NaN,NaN,NaN,nan
680,accept_score,9.895833,48.0,School C,Year 10,All,All,All,232.0,6.0,NaN,NaN,NaN,NaN,nan
977,accept_score,9.540541,37.0,School D,Year 10,All,All,All,232.0,6.0,NaN,NaN,NaN,NaN,nan
1274,accept_score,10.090909,33.0,School E,Year 10,All,All,All,232.0,6.0,NaN,NaN,NaN,NaN,nan
1571,accept_score,NaN,NaN,School F,Year 10,All,All,All,232.0,6.0,NaN,NaN,NaN,NaN,nan
1868,accept_score,10.540541,37.0,School G,Year 10,All,All,All,232.0,6.0,NaN,NaN,NaN,NaN,nan


## Set to NaN if not relevant

In [12]:
# Birth you age score is just average birth age and not needed as a "score"
# Overall count is the count of pupils in each group (not for specific variable)
mask = rag['variable'].isin(['birth_you_age_score', 'overall_count'])
nan_col = ['group_n', 'group_wt_mean', 'group_wt_std', 'lower', 'upper', 'rag']
rag.loc[mask, nan_col] = np.nan
rag[mask]

,variable,mean,count,school_lab,year_group_lab,gender_lab,fsm_lab,sen_lab,total_pupils,group_n,group_wt_mean,group_wt_std,lower,upper,rag
0,birth_you_age_score,8.105505,109.0,School A,All,All,All,All,667.0,NaN,NaN,NaN,NaN,NaN,NaN
33,birth_you_age_score,8.737288,59.0,School A,Year 8,All,All,All,372.0,NaN,NaN,NaN,NaN,NaN,NaN
66,birth_you_age_score,7.360000,50.0,School A,Year 10,All,All,All,282.0,NaN,NaN,NaN,NaN,NaN,NaN
99,birth_you_age_score,9.187500,16.0,School A,All,Girl,All,All,98.0,NaN,NaN,NaN,NaN,NaN,NaN
132,birth_you_age_score,8.900000,20.0,School A,All,Boy,All,All,120.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2137,overall_count,NaN,19.0,School G,All,Boy,All,All,128.0,NaN,NaN,NaN,NaN,NaN,NaN
2138,overall_count,NaN,57.0,School G,All,All,FSM,All,361.0,NaN,NaN,NaN,NaN,NaN,NaN
2139,overall_count,NaN,37.0,School G,All,All,Non-FSM,All,323.0,NaN,NaN,NaN,NaN,NaN,NaN
2140,overall_count,NaN,38.0,School G,All,All,All,SEN,272.0,NaN,NaN,NaN,NaN,NaN,NaN


## Add labels

In [13]:
var_lab = {
    'autonomy_score': 'Autonomy',
    'life_satisfaction_score': 'Life satisfaction',
    'optimism_score': 'Optimism',
    'wellbeing_score': 'Psychological wellbeing',
    'esteem_score': 'Self-esteem',
    'stress_score': 'Stress and coping',
    'appearance_score': 'Feelings around appearance',
    'negative_score': 'Negative affect',
    'lonely_score': 'Loneliness',
    'support_score': 'Supporting own wellbeing',
    'sleep_score': 'Sleep',
    'physical_score': 'Physical activity',
    'free_like_score': 'Free time',
    'media_score': 'Social media use',
    'places_score': 'Places to go and things to do',
    'talk_score': 'Talking about feelings',
    'accept_score': 'Acceptance',
    'school_belong_score': 'School connection',
    'staff_relationship_score': 'Support from staff',
    'home_relationship_score': 'Support from parents/carers',
    'home_happy_score': 'Home environment',
    'local_env_score': 'Local environment',
    'discrim_score': 'Discrimination',
    'belong_local_score': 'Local connection',
    'wealth_score': 'Relative wealth',
    'future_score': 'Future opportunities',
    'climate_score': 'Climate change',
    'social_score': 'Support from friends',
    'bully_score': 'Bullying'
}

In [14]:
# Add label column
rag['variable_lab'] = rag['variable'].map(var_lab)

# View the labels
rag[['variable', 'variable_lab']].drop_duplicates()

,variable,variable_lab
0,birth_you_age_score,NaN
1,autonomy_score,Autonomy
2,life_satisfaction_score,Life satisfaction
3,optimism_score,Optimism
4,wellbeing_score,Psychological wellbeing
5,esteem_score,Self-esteem
6,stress_score,Stress and coping
7,appearance_score,Feelings around appearance
8,negative_score,Negative affect
9,lonely_score,Loneliness


## Add descriptions

In [15]:
describe = {
    'autonomy_score': '''How 'in control' young people feel of their life''',
    'life_satisfaction_score': 'How satisfied young people feel with their life',
    'optimism_score': '''Young people's hopefulness and confidence for the future''',
    'wellbeing_score': 'How positive and generally happy young people feel regarding their life',
    'esteem_score': 'How much young people value themselves',
    'stress_score': 'Managing stress levels and coping with difficulties',
    'appearance_score': '''Young people's feelings around the way that they look''',
    'negative_score': 'The frequency with which young people experience emotional difficulties',
    'lonely_score': 'How often young people feel lonely',
    'support_score': '''Young people's knowledge on supporting themselves and looking for advice''',
    'sleep_score': 'How much sleep young people get',
    'physical_score': 'How physically active young people are',
    'free_like_score': 'How often young people can do things that they like in their free time',
    'media_score': 'How much time young people spend on social media',
    'places_score': '''Whether young people feel there are places to go and things to do in their free time''',
    'talk_score': '''How positively/negatively young people feel about talking with others about feeling down''',
    'accept_score': 'Whether young people feel accepted by different groups of people in their life',
    'school_belong_score': 'Feelings of belonging at school',
    'staff_relationship_score': 'The support received from adults at school',
    'home_relationship_score': 'The support received from adults at home',
    'home_happy_score': '''Young people's feelings regarding the home that they live in''',
    'local_env_score': 'How young people feel regarding the area where they live',
    'discrim_score': 'Whether young people feel discriminated against',
    'belong_local_score': '''Young people's feelings of belonging in their local area''',
    'wealth_score': 'Whether young people feel their family is richer, poorer or the same as their friends',
    'future_score': 'How young people feel regarding the future options for work, education or training in their local area',
    'climate_score': 'Worries regarding climate change',
    'social_score': 'The support young people receive from their peers',
    'bully_score': 'The frequency with which young people experience different types of bullying'
}

In [16]:
rag['description'] = rag['variable'].map(describe)
rag[['variable_lab', 'description']].drop_duplicates()

,variable_lab,description
0,NaN,NaN
1,Autonomy,How 'in control' young people feel of their life
2,Life satisfaction,How satisfied young people feel with their life
3,Optimism,Young people's hopefulness and confidence for ...
4,Psychological wellbeing,How positive and generally happy young people ...
5,Self-esteem,How much young people value themselves
6,Stress and coping,Managing stress levels and coping with difficu...
7,Feelings around appearance,Young people's feelings around the way that th...
8,Negative affect,The frequency with which young people experien...
9,Loneliness,How often young people feel lonely


## Save as csv file

In [17]:
rag.to_csv(os.path.join(paths.survey, paths.rag), index=False)